In [1]:
import pandas as pd

# Load the dataset to examine its structure
file_path = 'Agmarknet_Price_Report.csv'
data = pd.read_csv(file_path)

# Display basic information about the dataset
data.info(), data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108796 entries, 0 to 108795
Data columns (total 3 columns):
 #   Column                                                                             Non-Null Count   Dtype  
---  ------                                                                             --------------   -----  
 0   Min                                                                                108796 non-null  object 
 1   Max                                                                                0 non-null       float64
 2   Modal PriceLasalgaon from 08-Mar-2005To18-Sep-2024 (Total-4945) (Total-4945)<div>  0 non-null       float64
dtypes: float64(2), object(1)
memory usage: 2.5+ MB


(None,
                                                  Min  Max  \
 0  \t<table class="tableagmark_new" cellspacing="...  NaN   
 1  \t\t<tr style="color:White;background-color:Da...  NaN   
 2  \t\t\t<th scope="col">Sl no.</th><th scope="co...  NaN   
 3                                      \t\t</tr><tr>  NaN   
 4                                         \t\t\t<td>  NaN   
 
    Modal PriceLasalgaon from 08-Mar-2005To18-Sep-2024 (Total-4945) (Total-4945)<div>  
 0                                                NaN                                  
 1                                                NaN                                  
 2                                                NaN                                  
 3                                                NaN                                  
 4                                                NaN                                  )

In [2]:
# Try to clean the dataset by extracting relevant content from the 'Min' column
# It seems the data is mixed with HTML, so we'll look for patterns

# Preview the first few rows to determine the content
data_preview = data['Min'].head(20)
data_preview


0     \t<table class="tableagmark_new" cellspacing="...
1     \t\t<tr style="color:White;background-color:Da...
2     \t\t\t<th scope="col">Sl no.</th><th scope="co...
3                                         \t\t</tr><tr>
4                                            \t\t\t<td>
5                                                     1
6                                                 </...
7                                         <span id="...
8                                             </td><td>
9                                         <span id="...
10                                            </td><td>
11                                        <span id="...
12                                            </td><td>
13                                        <span id="...
14                                            </td><td>
15                                        <span id="...
16                                            </td><td>
17                                        <span 

In [3]:
from bs4 import BeautifulSoup

# Function to clean HTML-like content and extract text
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply the cleaning function to the 'Min' column to extract meaningful data
data_cleaned = data['Min'].apply(clean_html)

# Display the cleaned version of the first few rows to inspect
data_cleaned.head(20)


0                                                      
1                                                      
2      Sl no.District NameMarket NameCommodityVariet...
3                                                      
4                                                      
5                                                     1
6                                                      
7                                                Nashik
8                                                      
9                                             Lasalgaon
10                                                     
11                                                Onion
12                                                     
13                                                Other
14                                                     
15                                                  FAQ
16                                                     
17                                              

In [4]:
# Extract relevant fields (District Name, Market Name, Commodity, Variety, etc.) from the cleaned data
extracted_data = []

# We will iterate through the cleaned data and organize it in a structured format
temp_row = {}
for row in data_cleaned:
    if row.strip().isdigit():  # If it's a serial number, it indicates a new row of data
        if temp_row:
            extracted_data.append(temp_row)  # Append the previous row
        temp_row = {}  # Reset for new row
    elif 'District Name' in row:
        temp_row['District Name'] = row.split('District Name')[-1].strip()
    elif 'Market Name' in row:
        temp_row['Market Name'] = row.split('Market Name')[-1].strip()
    elif 'Commodity' in row:
        temp_row['Commodity'] = row.split('Commodity')[-1].strip()
    elif row.isdigit():
        temp_row['Price'] = int(row)

# Convert the structured data into a DataFrame
extracted_df = pd.DataFrame(extracted_data)

# Show the first few rows of the cleaned and organized data
extracted_df.head()


,District Name
0,Market NameCommodityVarietyGradeMin Price (Rs....


SARIMA Model

In [7]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# Load your dataset
df = pd.read_csv('cleaned_onion_price_data.csv', index_col='Date', parse_dates=True)

# Split the data into training and testing
train = df[:-30]
test = df[-30:]

# Build the SARIMA model
sarima_model = SARIMAX(train['Price'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_result = sarima_model.fit()

# Forecast
forecast = sarima_result.predict(start=len(train), end=len(train) + len(test) - 1)

# Evaluate
rmse = mean_squared_error(test['Price'], forecast, squared=False)
print(f'RMSE: {rmse}')


ValueError: 'Date' is not in list

Random Forest Regressor

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define features (X) and target (y)
X = df[['feature1', 'feature2']]  # Replace with actual features
y = df['Price']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')


NameError: name 'df' is not defined

XG Boost

In [9]:
# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import matplotlib.pyplot as plt

# Load your dataset
df = pd.read_csv('cleaned_onion_price_data.csv')

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

X = df.drop(columns=['Price'])
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=5)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)

print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')

# Plot the actual vs predicted values
plt.figure(figsize=(10, 5))
plt.plot(y_test.index, y_test.values, label='Actual Prices', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted Prices', color='red')
plt.title('Actual vs Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


KeyError: 'Date'